In [ ]:
import sys
print(sys.prefix)

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import quantstats as qs
import os

from datetime import timedelta
from datetime import datetime
from dateutil.relativedelta import *
from pandas.tseries.offsets import MonthEnd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# from mpt import portfolio_return 
import quantstats as qs
import dataframe_image as dfi

In [ ]:
exp_path = Path(r"C:\Users\Mathiass\Documents\Projects\master-thesis\logs\old\clustersmall5")

In [ ]:
path_portfolios = exp_path/"portfolios"
dfs = []
for file in Path.iterdir(path_portfolios):
    try:
        df = pd.read_csv(path_portfolios/file, parse_dates=["date"], index_col="date")
    except PermissionError as err:
        raise PermissionError("The 'portfolios' subfolder must not contain directories.") from err
        # from err necessary for chaining
    dfs.append(df["option_ret"].rename(file.name[7:-4])) #rename Series to 'class0', 'class1', etc.
dfs = pd.concat(dfs, axis=1) # Series names -> column names

In [ ]:
dfs

In [ ]:
import yfinance as yf

In [ ]:
exp_dir = Path(r"C:\Users\Mathiass\Documents\Projects\master-thesis\logs\old\clustersmall5")

In [ ]:
agg_df = pd.read_csv(exp_dir/"portfolios"/"class0.csv", parse_dates=["date"], index_col="date")

In [ ]:
agg_df

In [ ]:
end = agg_df.iloc[-1].name.to_pydatetime() + timedelta(days=1)
agg_df.iloc[-1].name.to_pydatetime() + timedelta(days=1)

In [ ]:
# tickers = ["^SP500TR", "MSFT", "TSLA", "AAPL", "NFLX"]
tickers = ["^SP500TR"]

In [ ]:
# sp = yf.Ticker(tickers)

In [ ]:
# df1 = sp.history(end="2021-12-01", period="max", auto_adjust=False, prepost=False)

In [ ]:
# df1

In [ ]:
# end = datetime(2021, 12, 1)

In [ ]:
end

In [ ]:
df = yf.download(tickers=tickers, end=end, auto_adjust=False, prepost=False)

In [ ]:
df

In [ ]:
df_monthend = df.iloc[df.reset_index(drop=False).groupby(df.index.to_period("M"))["Date"].idxmax()]

In [ ]:
df_monthend.index[-23:]

In [ ]:
df_monthend.index = df_monthend.index + MonthEnd(0)
df_monthend.index[-23:]

In [ ]:
df_monthend[-106:]["Close"]

In [ ]:
agg_df

In [ ]:
df_monthend.loc[agg_df.index]["Adj Close"]

In [ ]:
df_monthend.iloc[-len(agg_df.index) - 1:]["Adj Close"]

In [ ]:
agg_df.index

In [ ]:
df_percent = df_monthend[-len(agg_df.index) - 1:].pct_change()

In [ ]:
sp500 = df_percent.iloc[1:]["Adj Close"]

In [ ]:
opt_ret = agg_df["option_ret"]

In [ ]:
# only intercept or with regressor:
X = np.array([np.ones_like(sp500), sp500]).T

In [ ]:
alphabeta = np.linalg.inv(X.T@X)@X.T@opt_ret

In [ ]:
alphabeta

### Standard Errors of OLS

In [ ]:
predict = X@alphabeta

In [ ]:
predict

In [ ]:
residuals = opt_ret - predict

In [ ]:
residuals

In [ ]:
# Sample variance of errors.
sigma_squared_hat = (residuals.T @ residuals) / (X.shape[0] - X.shape[1])

In [ ]:
sigma_squared_hat

In [ ]:
# Var-Covariance matrix of beta hat.
var_beta_hat = np.linalg.inv(X.T@X) * sigma_squared_hat

In [ ]:
np.linalg.inv(X.T@X)

In [ ]:
var_beta_hat

In [ ]:
# Coefficients
alphabeta

In [ ]:
# Standard errors of intercept and regressors
for var in range(len(var_beta_hat)):
        se = var_beta_hat[var, var] ** 0.5
        print(se)

In [ ]:
# P value for t score
import scipy.stats

#find p-value for two-tailed test
scipy.stats.t.sf(abs(1.725), df=165)*2

### Check with statsmodels:

In [ ]:
import statsmodels.api as sm
ols = sm.OLS(opt_ret, X)
ols_result = ols.fit()
# ols_result = ols.fit(cov_type="HAC", cov_kwds={"maxlags": 12})
ols_result.summary()

In [ ]:
# Results to pandas dataframe.
# results_as_html = ols_result.summary().tables[1].as_html()
# pd.read_html(results_as_html, header=0, index_col=0)[0]

In [ ]:
result_df = ols_result.summary2(alpha=0.05).tables[1]
result_df

### Intercept only regression == significance of mean above 0 ?

In [ ]:
# Intercept only regression == mean?
np.mean(opt_ret)

In [ ]:
# Sample variance of residuals
sample_variance = 1/(len(sp500) - 1)* np.sum((opt_ret - np.mean(opt_ret))**2)
sample_variance

In [ ]:
# equal to
np.var(opt_ret, ddof=1)

In [ ]:
# SE = SD / sqrt(n)
np.sqrt(sample_variance) / np.sqrt(len(sp500))

### Example Intercept only: Binary val_bal_acc significance above 0.5

In [ ]:
# binary small nn
balacc = np.array([0.5,0.532052755355835,0.5655055046081543,0.5297303795814514,0.5083361864089966,0.5356613397598267,0.552789032459259,0.5454691052436829,0.5263364911079407,0.5344380140304565,0.5364874601364136,0.5388208627700806,0.5325674414634705,0.5308846235275269])

In [ ]:
# Subtract Null hypothesis
balacc -= 0.5

In [ ]:
X = np.ones_like(balacc)

In [ ]:
assert balacc.shape == X.shape

In [ ]:
balacc

In [ ]:
len(balacc)

In [ ]:
X = X.reshape(-1, 1)

In [ ]:
balacc = balacc.reshape(-1, 1)

In [ ]:
np.linalg.inv(X.T@X)@X.T@balacc

In [ ]:
predict = X@np.linalg.inv(X.T@X)@X.T@balacc

In [ ]:
residuals = balacc - predict

In [ ]:
# Standard Error of mean
se = (np.linalg.inv(X.T@X) * residuals.T@residuals / (X.shape[0] - X.shape[1])) ** 0.5
se

In [ ]:
ols = sm.OLS(balacc, X)
ols_result = ols.fit()
# ols_result = ols.fit(cov_type="HAC", cov_kwds={"maxlags": 12})
ols_result.summary()

In [ ]:
np.mean(balacc)

In [ ]:
np.std(balacc, ddof=1) / np.sqrt(len(balacc))

In [ ]:
# If balacc not subtracted by null hypothesis, do this:
(np.mean(balacc) - 0.5) / (np.std(balacc, ddof=1) / np.sqrt(len(balacc)))

### Example Intercept only: Portfolio returns significantly above 0 ?

In [ ]:
opt_ret= [0.440820444, -0.518864416, 0.728927879, -0.040891163, 0.163143079, 0.467870532, -0.040914468, -0.052467521, -0.079048695, 0.031793878, -0.031312123, 0.08498215, -0.00616937, -0.083816107, -0.029022221, -0.01706333, 0.010316496, -0.039103077, -0.021999441, -0.005304246, 0.01883039, 0.028072462, -0.030337784
]

In [ ]:
# mean
np.mean(opt_ret)

In [ ]:
# standard error
np.std(opt_ret, ddof=1) / np.sqrt(len(opt_ret))

In [ ]:
# t-stat
np.mean(opt_ret) / (np.std(opt_ret, ddof=1) / np.sqrt(len(opt_ret)))

In [ ]:
# P value for t score:
import scipy.stats
#find p-value for two-tailed test
scipy.stats.t.sf(abs(0.8618805965877891), df=len(opt_ret)-1)*2

In [ ]:
# import pandas_datareader.data as web

In [ ]:
# df2 = web.DataReader(tickers, data_source='yahoo', end=end)

In [ ]:
# yf.pdr_override()

In [ ]:
# data = web.get_data_yahoo(tickers, end=end, auto_adjust=False, prepost=False)

In [ ]:
# data

In [ ]:
# data["Adj Close"][:10]

In [ ]:
import requests

r = requests.get('https://api.github.com/events')

In [ ]:
r.json()

In [ ]:
bad_r = requests.get('https://httpbin.org/status/200')

In [ ]:
bad_r.headers

In [ ]:
sns.set_style("whitegrid") # style must be one of white, dark, whitegrid, darkgrid, ticks
sns.set_context('paper', font_scale=2.0) # set fontsize scaling for labels, axis, legend, automatically moves legend

In [ ]:
path = Path(r"C:\Users\Mathiass\Documents\Projects\master-thesis\logs\tune\nn_loops\20220808101241\portfolios")

In [ ]:
# (path/'test').mkdir()

In [ ]:
dfs = []
for file in os.listdir(path):
    df = pd.read_csv(path/file, parse_dates=["date"], index_col="date")
    dfs.append(df["option_ret"].rename(file[:-4]))
#     dfs.append(df)

In [ ]:
dfs = pd.concat(dfs, axis=1)

In [ ]:
dfs


In [ ]:
dfs2 = []
for file in Path.iterdir(path):
    df = pd.read_csv(path/file, parse_dates=["date"], index_col="date")
    dfs2.append(df["option_ret"].rename(file.name[7:-4]))

In [ ]:
for i in os.listdir(path):
    print(i)

In [ ]:
for f in Path.iterdir(path):
    print(f.name)

In [ ]:
# dfs_idx = dfs_idx.insert(0, dfs_idx[0] - relativedelta(months=+1))

# dfs_idx[0]

# df = pd.concat([pd.DataFrame([0], index=[dfs_idx[0]]), df])
# df.columns = ["option_ret"]

In [ ]:
# df to series
# df = df.squeeze()

In [ ]:
dfs_cumprod = (1. + dfs).cumprod()
dfs_cumprod.plot(figsize=(15, 10), alpha=1.0, linewidth=2.5, ylabel="Portfolio Value")
plt.tight_layout() # remove whitespace around plot
path = os.path.join("results", datetime.now().strftime("%Y%m%d%H%M%S"))
os.makedirs(path, exist_ok=True)
plt.savefig(os.path.join(path, "plot.png"))

In [ ]:
path

In [ ]:
# path = os.path.join("results", datetime.now().strftime("%Y%m%d%H%M%S"))


# os.makedirs(path, exist_ok=True)
# plt.savefig(os.path.join(path, "plot.png"))

In [ ]:
(np.log(dfs+1)).mean()

In [ ]:
(1 + dfs).prod()

In [ ]:
# Arithmetic mean
dfs.mean()

In [ ]:
# Geometric mean
((dfs+1).prod() ** (1 / len(dfs)) - 1)

In [ ]:
# annualized arithmetic mean
(dfs.mean() * 12)

In [ ]:
# annualized vol
qs.stats.volatility(dfs, periods=12)

In [ ]:
# check volatility
vol = dfs.std(ddof=1) * np.sqrt(12)
dfs.std(ddof=1) * np.sqrt(12)

##### # CAGR OF QUANTSTATS IS WRONLY CALCULATED FOR MONTHLY RETURNS (subtracts 1 from lenght of total months, assum. daily prices...)

In [ ]:
# Wrong. cum. annualized growth rate
qs.stats.cagr(dfs, compounded=True)

In [ ]:
# Wrong
cagr = (dfs+1).prod() ** (12/(len(dfs)-1)) - 1
print(cagr)

In [ ]:
cagr = (dfs+1).prod() ** (12/(len(dfs)-1)) - 1

In [ ]:
cagr

In [ ]:
# Correct
cagr = (dfs+1).prod() ** (12/len(dfs)) - 1
print(cagr)

In [ ]:
# wrong, if mean is >1, will convert to pct_change... (class 1 is such a case)
qs.stats.sharpe(dfs, periods=12)

In [ ]:
# Check Sharpe Ratio
sharpe = dfs.mean(axis=0) / dfs.std() * np.sqrt(12)
dfs.mean(axis=0) / dfs.std() * np.sqrt(12)

In [ ]:
# Check Sharpe Ratio 2
(dfs.mean() * 12) / (dfs.std(ddof=1) * np.sqrt(12))

In [ ]:
# Sharpe Ratio with avg. geometric returns
((dfs+1).prod() ** (12/ len(dfs)) - 1) / (dfs.std(ddof=1) * np.sqrt(12))

In [ ]:
cagr / (dfs.std(ddof=1) * np.sqrt(12))

In [ ]:
cagr/vol

In [ ]:
(dfs.mean() * 12)

In [ ]:
(dfs.std(ddof=1) * np.sqrt(12))

In [ ]:
# Kurtosis
dfs.kurtosis(axis=0)

In [ ]:
kurt =qs.stats.kurtosis(dfs)
kurt

In [ ]:
# Skewness
skew = dfs.skew()
skew

In [ ]:
qs.stats.skew(dfs)

In [ ]:
dfsp = dfs.copy()

In [ ]:
# dfsp = pd.concat([pd.DataFrame([[0, 0]], columns=dfsp.columns), dfsp], ignore_index=False)

In [ ]:
dfsp.head()

In [ ]:
dfs.index[0] - MonthEnd(1)

In [ ]:
dfsp.loc[dfsp.index[0] - MonthEnd(1)] = [0] * dfsp.shape[1]

dfsp = dfsp.sort_index()

In [ ]:
dfsp

In [ ]:
prices =(1 + dfsp).cumprod()

In [ ]:
prices.head()

In [ ]:
 prices.expanding(min_periods=0).max().head()

In [ ]:
(prices / prices.expanding(min_periods=0).max())

In [ ]:
prices

In [ ]:
(prices / prices.expanding().max()).min()

In [ ]:
(prices / prices.expanding().max()).min() - 1

In [ ]:
# Max. DD is correctly calculated.
qs.stats.max_drawdown(dfs)

In [ ]:
qs.stats.drawdown_details(qs.stats.to_drawdown_series(dfsp))["long1short0"].sort_values(by="max drawdown", ascending=True)

In [ ]:
maxdd = (prices / prices.expanding().max()).min() - 1

In [ ]:
maxdd

In [ ]:
(prices / prices.expanding().max())

In [ ]:
maxdd["class0"]

In [ ]:
qs.stats.drawdown_details(qs.stats.to_drawdown_series(dfs))["class0"].sort_values(by="max drawdown", ascending=False)

In [ ]:
a = qs.stats.drawdown_details(qs.stats.to_drawdown_series(dfs))["class0"].sort_values(by="max drawdown", ascending=True)["max drawdown"].iloc[0] / 100

In [ ]:
a == maxdd[0]

In [ ]:
qs.stats.drawdown_details(qs.stats.to_drawdown_series(dfs))["class0"].sort_values(by="max drawdown", ascending=True)["days"].iloc[0]

In [ ]:
qs.stats.drawdown_details(qs.stats.to_drawdown_series(dfs))["class1"].sort_values(by="max drawdown", ascending=True)["days"].iloc[0]

In [ ]:
dfsp

In [ ]:
dd_days=[]
for columname in dfsp.columns:
    dd_days.append(qs.stats.drawdown_details(qs.stats.to_drawdown_series(dfsp))[columname].sort_values(by="max drawdown", ascending=True)["days"].iloc[0])
dd_days = pd.Series(dd_days, index=dfsp.columns, name="Max DD days")
dd_days = dd_days.apply(lambda x: f'{x: .0f}')

In [ ]:
dd_days

In [ ]:
maxdd.abs()

In [ ]:
# with wrong cagr
qs.stats.cagr(dfs) / qs.stats.max_drawdown(dfs).abs()

In [ ]:
# with wrong cagr
qs.stats.cagr(dfs) / maxdd.abs()

In [ ]:
# with wrong cagr
qs.stats.calmar(dfs)

In [ ]:
# with correct cagr
cagr / qs.stats.max_drawdown(dfs).abs()

In [ ]:
# with correct cagr
calmar = cagr / maxdd.abs()

In [ ]:
# cumulative return
cumr = (1+dfs).prod() - 1

In [ ]:
# alpha beta

In [ ]:
# only intercept or with regressor:
X = np.array([np.ones_like(sp500), sp500]).T

In [ ]:
alphabeta = np.linalg.inv(X.T@X)@X.T@dfs

In [ ]:
alphabeta

In [ ]:
# alpha (annualized), beta
alpha = alphabeta.iloc[0, :] * 12
beta = alphabeta.iloc[1, :]

In [ ]:
# mean (annualized)
mean = dfs.mean() * 12

In [ ]:
mean

In [ ]:
perfstats = [cumr, cagr, mean, vol, sharpe, maxdd, dd_days, calmar, skew, kurt, alpha, beta] # then here.
# perfstats = [vol, cagr, sharpe] # then here.

# save results
perfstats = pd.concat(perfstats, axis=1)
perfstats.to_csv(os.path.join(path, "perfstats.csv"))    
dfi.export(perfstats, os.path.join(path, "perfstats.png"))

# export latex code for table
with open(os.path.join(path, "latex.txt"), "w") as text_file:
    text_file.write(perfstats.to_latex(float_format="%.2f"))
    text_file.write("\n")
    text_file.write("% Same table transposed:\n")
    text_file.write("\n")
#     text_file.write(perfstats.T.to_latex(float_format="%.2f"))

In [ ]:
path

In [ ]:
qs.stats.volatility(df_full.iloc[:, 1:], periods=12)

In [ ]:
(df.index[-1] - df.index[0]).days / 365

In [ ]:
23 / 12

In [ ]:
df.index

In [ ]:
12/23

In [ ]:
((1 + df).prod() ** (12/23))

In [ ]:
qs.stats.cagr(df)

In [ ]:
qs.stats.cagr(df)

In [ ]:
qs.stats.volatility(df, periods=12)

In [ ]:
qs.stats.sharpe(df, periods=12)

In [ ]:
0.763922 / 0.942529

In [ ]:
# mean is annualized * 12, std with sqrt(12)
df.mean() * 12 / (df.std() * np.sqrt(12))

In [ ]:
qs.stats.volatility(df, periods=12)

In [ ]:
df.mean() * 12 / (df.std())

In [ ]:
qs.plots.snapshot(df)

In [ ]:
df.mean() / df.std(ddof=1) * np.sqrt(252)

In [ ]:
qs.reports.plots(df, "full", periods_per_year=12)

In [ ]:
qs.reports.metrics(df, mode="basic", periods_per_year=12)

In [ ]:
help(qs.plots.snapshot)

In [ ]:
# annualized std
np.std(df, ddof=1) * np.sqrt(12)

In [ ]:
qs.reports.html(df, "SPY", periods_per_year=12)

In [ ]:
# a = [1,2,3]

In [ ]:
dfs.mean()